## Getting started

Import Brightway2 functionality

In [1]:
from brightway2 import *

Create new project

In [2]:
projects.current = "underspecification"

Install base data

In [3]:
bw2setup()

Creating default biosphere

Applying strategy: drop_unspecified_subcategories

Writing activities to SQLite3 database:
0%                          100%
[##############################] | ETA[sec]: 0.000 
Total time elapsed: 0.326 sec



Title: Writing activities to SQLite3 database:
  Started: 08/25/2015 13:38:21
  Finished: 08/25/2015 13:38:22
  Total time elapsed: 0.326 sec
  CPU %: 96.900000
  Memory %: 0.263476
Created database: biosphere3
Creating default LCIA methods

Applying strategy: set_biosphere_type
Applying strategy: drop_unspecified_subcategories
Applying strategy: link_iterable_by_fields
Wrote 692 LCIA methods with 170915 characterization factors
Creating core data migrations



## Importing ecoinvent 3.1

Import ecoinvent 3.1 cutoff, but don't do any data manipulation yet

In [4]:
ei31cutoff = SingleOutputEcospold2Importer(
    "/Users/cmutel/Documents/LCA Documents/Ecoinvent/3.1/cutoff/datasets",
    "ecoinvent 3.1 cutoff"
)

Extracting ecospold2 files:
0%                          100%
[##############################] | ETA[sec]: 0.000 | Item ID: fff527b1-0fe4-4
Total time elapsed: 71.115 sec


Title: Extracting ecospold2 files:
  Started: 08/25/2015 13:39:06
  Finished: 08/25/2015 13:40:17
  Total time elapsed: 71.115 sec
  CPU %: 87.600000
  Memory %: 2.952015
Extracted 11301 datasets in 72.01 seconds


In [14]:
isic_codes = {dict(ds['classifications']).get('ISIC rev.4 ecoinvent') for ds in ei31cutoff.data}
isic_codes.discard(None)
sorted(isic_codes)

['0111:Growing of cereals (except rice), leguminous crops and oil seeds',
 '0112:Growing of rice',
 '0113:Growing of vegetables and melons, roots and tubers',
 '0114:Growing of sugar cane',
 '0116:Growing of fibre crops',
 '0119:Growing of other non-perennial crops',
 '0121:Growing of grapes',
 '0122:Growing of tropical and subtropical fruits',
 '0123:Growing of citrus fruits',
 '0124:Growing of pome fruits and stone fruits',
 '0125:Growing of other tree and bush fruits and nuts',
 '0126:Growing of oleaginous fruits',
 '0129:Growing of other perennial crops',
 '012:Growing of perennial crops',
 '0130:Plant propagation',
 '0141:Raising of cattle and buffaloes',
 '0144:Raising of sheep and goats',
 '0145:Raising of swine/pigs',
 '014:Animal production',
 '0161:Support activities for crop production',
 '0162:Support activities for animal production',
 '0163:Post-harvest crop activities',
 '0164:Seed processing for propagation',
 '01:Crop and animal production, hunting and related service 

In [23]:
group_by_region = {}

for ds in ei31cutoff.data:
    code, region = dict(ds['classifications']).get('ISIC rev.4 ecoinvent'), ds['location']
    if code:
        group_by_region[(code, region)] = group_by_region.get((code, region), 0) + 1

In [24]:
sorted([(key, value) for key, value in group_by_region.items() if value > 1])

[(('0111:Growing of cereals (except rice), leguminous crops and oil seeds',
   'BR'),
  2),
 (('0111:Growing of cereals (except rice), leguminous crops and oil seeds',
   'CH'),
  44),
 (('0111:Growing of cereals (except rice), leguminous crops and oil seeds',
   'CN'),
  2),
 (('0111:Growing of cereals (except rice), leguminous crops and oil seeds',
   'DE'),
  4),
 (('0111:Growing of cereals (except rice), leguminous crops and oil seeds',
   'ES'),
  4),
 (('0111:Growing of cereals (except rice), leguminous crops and oil seeds',
   'FR'),
  4),
 (('0111:Growing of cereals (except rice), leguminous crops and oil seeds',
   'GLO'),
  44),
 (('0111:Growing of cereals (except rice), leguminous crops and oil seeds',
   'RER'),
  2),
 (('0111:Growing of cereals (except rice), leguminous crops and oil seeds',
   'RoW'),
  37),
 (('0111:Growing of cereals (except rice), leguminous crops and oil seeds',
   'US'),
  4),
 (('0113:Growing of vegetables and melons, roots and tubers', 'CH'), 4),
 

In [27]:
sum(group_by_region.values()), len(group_by_region), len(ei31cutoff.data)

(11184, 1117, 11301)

In [32]:
[x for x in ei31cutoff.data if x['location'] == 'DK' and 'market for electricity, high' in x['name']]

[{'activity': 'ac3c5ebf-a3d2-4bda-92ce-80af2f73999b',
  'activity type': 'market activity',
  'classifications': [('EcoSpold01Categories', 'electricity/supply mix'),
   ('ISIC rev.4 ecoinvent',
    '3510:Electric power generation, transmission and distribution')],
  'comment': 'This dataset describes the electricity available on the high voltage level in this country. This is done by showing the transmission of 1kWh electricity at high voltage.\n\n\nTechnology:  Average technology used to transmit and distribute electricity. Includes underground and overhead lines. Electricity production according to related  technology datasets.\n\nDefinition of the voltage levels: \n - High voltage level above 24 kV (large scale industry)\n - Medium voltage level between 1 kV and 24 kV (medium to small scale industry, service sector\nand public buildings)\n - Low voltage level below 1 kV (Households)\nTime period:  The annual production volumes of all electricity production datasets in ecoinvent vers

In [33]:
11301 - 11184

117